In [54]:
# load instructed-tuned ckpt
from inference_module import SFMGenerator
from tqdm import tqdm
from collections import defaultdict

ckpt_home = '/home/yinxia/blob1.v2/yinxia/scigpt/7bv3/overall_instruct_20240430/run2/global_step24128'
generator = SFMGenerator(ckpt_home)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.
Loading checkpoint shards: 100%|██████████| 2/2 [00:27<00:00, 13.65s/it]


layer0 embed_tokens.weight torch.Size([40014, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size([4096])
02 post_attention_layernorm.weight torch.Size([4096])
03 self_attn.q_proj.weight torch.Size

In [55]:
# define some post processors
from rdkit import Chem, RDLogger
RDLogger.DisableLog('rdApp.*')

def parse_smiles(smi):
    r = smi.replace('<mol>', '').replace('</mol>', '')
    r = r.replace('<m>', '')
    r = r.replace(' ', '')
    m = Chem.MolFromSmiles(r)
    if m is None:
        return None

    return Chem.MolToSmiles(m)

def parse_protein(fasta):
    if '</protein>' not in fasta:
        return None
    fasta = fasta.replace('<protein>', '').replace('</protein>', '')
    fasta = fasta.replace('<a>', '')
    fasta = fasta.replace(' ', '')
    return fasta.strip()

In [56]:
from collections import defaultdict
query = 'generate an inhibitor for HIV'

beam_outs = []
random_outs = defaultdict(int)

# beam seaarch outputs
ret = generator.chat(query, do_sample=False)
for r in ret:
    s = parse_smiles(r)
    if s is not None:
        beam_outs.append(s)

print('beam search output')
for s in beam_outs:
    print(s)

# random s
for _ in range(10):
    ret = generator.chat(query, do_sample=True)
    for r in ret:
        s = parse_smiles(r)
        if s is not None:
            random_outs[s] += 1

sorted_random_outs = sorted(random_outs.items(), key=lambda x: x[1], reverse=True)
print('random sample search output')
for s in sorted_random_outs:
    print(s)

beam search output
Cc1cccc(C)c1OCC(=O)N[C@@H](Cc1ccccc1)[C@@H](O)C[C@H](Cc1ccccc1)NC(=O)[C@H](C(C)C)N1CCCNC1=O
CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)c1ccc2ccccc2n1
Cc1cc(C#N)cc(C)c1Oc1nc(Nc2ccc(C#N)cc2)nc(N)c1Br
Cc1ccnc2c1NC(=O)c1cccnc1N2C1CC1
random sample search output
('Cc1cccc(C)c1OCC(=O)N[C@@H](Cc1ccccc1)[C@@H](O)C[C@H](Cc1ccccc1)NC(=O)[C@H](C(C)C)N1CCCNC1=O', 7)
('Cc1ccnc2c1NC(=O)c1cccnc1N2C1CC1', 3)
('Cc1cc(C#N)cc(C)c1Oc1nc(Nc2ccc(C#N)cc2)nc(N)c1Br', 3)
('CC(C)(C)NC(=O)[C@@H]1CN(Cc2cccnc2)CCN1C[C@@H](O)C[C@@H](Cc1ccccc1)C(=O)N[C@H]1c2ccccc2C[C@H]1O', 2)
('CC(C)c1nc(CN(C)C(=O)N[C@H](C(=O)N[C@@H](Cc2ccccc2)[C@H](O)CN2CCCCC2)C(C)(C)C)cs1', 2)
('CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)c1ccc2ccccc2n1', 1)
('CC(C)(C)C(=O)OCOP(=O)(COCCn1cnc2c(N)ncnc21)OCOC(=O)C(C)(C)C', 1)
('CC(C)Nc1cccnc1N1CCN(C(=O)c2cc3cc(NS(C)(=O)=O)ccc3[nH]2)CC1', 1)
('CC(C)OC(=O)[C@H](C)N[P@](=O)(CO[C@H]

In [57]:
prop = 'QED' 
# You can replace $pror to any property you want, because we use GPT4 to extract the data

from rdkit.Chem.QED import qed

query_mol = r"CC(=O)OC1=CC=CC=C1C(=O)O"
s = Chem.MolFromSmiles(query_mol)
s2 = Chem.MolToSmiles(s)

query = f'improve the {prop} of {s2}'
filtered_mol = []
ret = generator.chat(query, do_sample=False)
for r in ret:
    s = parse_smiles(r)
    if s:
        filtered_mol.append(s)

print('Results of generated cmpds')
for s in filtered_mol:
    m = Chem.MolFromSmiles(s)
    print(s, qed(m))

print('Results of the original cmpd')
m = Chem.MolFromSmiles(query_mol)
print(qed(m))

Results of generated cmpds
O=C(O)CCC(=O)O 0.5437714031598524
O=C(O)/C=C/C(=O)O 0.4859839100851221
O=C(O)CCCCCCCCC(=O)O 0.5622977280227048
O=C(O)CCCC(=O)O 0.5774203601545895
Results of the original cmpd
0.5501217966938848


In [58]:
query_mol = r"C2=NC(=NC=C2)NC3=CC=C(C=C3)N4CCOCC4"
m = Chem.MolFromSmiles(query_mol)
s2 = Chem.MolToSmiles(m)

query = 'Improve the oral bioactivity of <mol>' + s2 + '</mol>'

filtered_mol = defaultdict(int)
for _ in tqdm(range(10)):
    ret = generator.chat(query, do_sample=True)
    for r in ret:
        s = parse_smiles(r)
        if s:
            filtered_mol[s] += 1

sorted_filtered_mol = sorted(filtered_mol.items(), key=lambda x: x[1], reverse=True)

100%|██████████| 10/10 [00:12<00:00,  1.26s/it]


In [59]:
query = 'Generate a membrane protein'
ret = generator.chat(query, do_sample=True, repetition_penalty=1.2, max_new_tokens=300)
for r in ret:
    r2 = parse_protein(r)
    if r2:
        print(r2)

MNRYIMPIIIILMVGCALCVMFIYYPALKYVMYWKAKSELSKWNIFHREWIKHGKN
MGKCCVTTCKNKQQYDCVCCNEGCAKKICECVNHKCVTKECERCK
MGSMSSWLRATHGQNEMRCHTCSPSASHLSRKLVPWAPGPGPVGPAGRAAARGRRARPWRAAAAVGPGRPRAAPRSPAPGPAGAPWRLWARPPAARAGGRGAGPESSSAGPRPRRRARRPRAPGAAAAGGGALRRPAEAAGARAEAGPDPAVRRRGGSGRRRRLHRDLALLVLEALAAAHDGERSAQQQTEEELEAEVAALRRALGRLRLVEEEARLVVLTPEAVRAELEPEPEPEVQEAWEDHLQEGARGRRDPGSC
MADKRGGGGRGGGGPRRGDRRPRDEQEEAAAAAGAGGEEAGGAAAGGVRAERRERGGRAAAGRRGREGGRRRRGAARRGRRRGRGRGRSRGGDGARARGG
